# Random forest
In the following we use the Random Forest method on the converted data from the descriptive_analysis.ipynb

Source Random Forest Model: https://towardsdatascience.com/random-forest-in-python-24d0893d51c0 Source Kfold Cross Validation on Random Forests: https://www.youtube.com/watch?v=gJo0uNL-5Qw Source Gradient Boosting on Random Forest: https://towardsdatascience.com/gradient-boosting-using-random-forests-for-application-on-the-new-york-taxi-fare-prediction-f6101c592bf9

The following Python code loads in the csv data and displays the structure of the data: 

In [1]:
# Pandas is used for data manipulation
import pandas as pd

# Read in data, set index to column 0, parse dates and display first 5 rows
test = pd.read_csv('./data/converted_test_2020_07_15_12-33-08.csv', index_col=0, parse_dates=['Flight_Date','Request_Date'])

training_features = pd.read_csv('./data/converted_train_2020_07_15_12-33-08.csv', index_col=0, parse_dates=['Flight_Date','Request_Date'])
training_features.head(5)

,Request_Date,Flight_Date,Duration_Time,Departure_hour,flight_unique_id,route_abb,flight_number,Price,Currency,min_future_price,...,min_future_price_in_eur,Request_Weekday,Request_timeofday,Departure_timeofday,flight_distance_km,Request_hour,Request_Date_Month,Flight_Date_Month,route_abb_from,route_abb_to
0,2019-06-03 11:00:00,2019-06-05 19:00:00,56.0,19:00,2019-06-05 FR 146,SXF-STN,FR 146,208.07,€,259.07,...,259.070000,Monday,Morning,Evening,907.2,11:00,6,6,SXF,STN
1,2019-06-03 23:00:00,2019-06-05 19:00:00,44.0,19:00,2019-06-05 FR 146,SXF-STN,FR 146,259.07,€,259.07,...,259.070000,Monday,Night,Evening,907.2,23:00,6,6,SXF,STN
2,2019-06-04 11:00:00,2019-06-05 19:00:00,32.0,19:00,2019-06-05 FR 146,SXF-STN,FR 146,259.07,€,259.07,...,259.070000,Tuesday,Morning,Evening,907.2,11:00,6,6,SXF,STN
3,2019-06-04 23:00:00,2019-06-05 19:00:00,20.0,19:00,2019-06-05 FR 146,SXF-STN,FR 146,259.07,€,259.07,...,259.070000,Tuesday,Night,Evening,907.2,23:00,6,6,SXF,STN
4,2019-06-03 11:00:00,2019-06-05 21:00:00,58.0,21:00,2019-06-05 FR 147,STN-SXF,FR 147,127.49,£,223.37,...,252.059401,Monday,Morning,Night,907.2,11:00,6,6,STN,SXF


In [2]:
# should be rounded in the file data conversion
test.round(2)
training_features.round(2)

,Request_Date,Flight_Date,Duration_Time,Departure_hour,flight_unique_id,route_abb,flight_number,Price,Currency,min_future_price,...,min_future_price_in_eur,Request_Weekday,Request_timeofday,Departure_timeofday,flight_distance_km,Request_hour,Request_Date_Month,Flight_Date_Month,route_abb_from,route_abb_to
0,2019-06-03 11:00:00,2019-06-05 19:00:00,56.0,19:00,2019-06-05 FR 146,SXF-STN,FR 146,208.07,€,259.07,...,259.07,Monday,Morning,Evening,907.2,11:00,6,6,SXF,STN
1,2019-06-03 23:00:00,2019-06-05 19:00:00,44.0,19:00,2019-06-05 FR 146,SXF-STN,FR 146,259.07,€,259.07,...,259.07,Monday,Night,Evening,907.2,23:00,6,6,SXF,STN
2,2019-06-04 11:00:00,2019-06-05 19:00:00,32.0,19:00,2019-06-05 FR 146,SXF-STN,FR 146,259.07,€,259.07,...,259.07,Tuesday,Morning,Evening,907.2,11:00,6,6,SXF,STN
3,2019-06-04 23:00:00,2019-06-05 19:00:00,20.0,19:00,2019-06-05 FR 146,SXF-STN,FR 146,259.07,€,259.07,...,259.07,Tuesday,Night,Evening,907.2,23:00,6,6,SXF,STN
4,2019-06-03 11:00:00,2019-06-05 21:00:00,58.0,21:00,2019-06-05 FR 147,STN-SXF,FR 147,127.49,£,223.37,...,252.06,Monday,Morning,Night,907.2,11:00,6,6,STN,SXF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83619,2019-08-01 11:00:00,2019-09-10 10:00:00,959.0,10:00,2019-09-10 FR 8543,SXF-STN,FR 8543,35.69,€,39.69,...,39.69,Thursday,Morning,Morning,907.2,11:00,8,9,SXF,STN
83620,2019-08-01 23:00:00,2019-09-10 10:00:00,947.0,10:00,2019-09-10 FR 8543,SXF-STN,FR 8543,46.83,€,39.69,...,39.69,Thursday,Night,Morning,907.2,23:00,8,9,SXF,STN
83621,2019-08-02 11:00:00,2019-09-10 10:00:00,935.0,10:00,2019-09-10 FR 8543,SXF-STN,FR 8543,46.83,€,39.69,...,39.69,Friday,Morning,Morning,907.2,11:00,8,9,SXF,STN
83622,2019-08-02 23:00:00,2019-09-10 10:00:00,923.0,10:00,2019-09-10 FR 8543,SXF-STN,FR 8543,39.69,€,39.69,...,39.69,Friday,Night,Morning,907.2,23:00,8,9,SXF,STN


In [3]:
#add the feature average min_future_price for all unique flight ids
#average_min_future_price = train_features.groupby('route_abb').min_future_price_in_eur.mean()
#average_min_future_price.round(2)

In [4]:
# Show how many entries and columns, and if there are null-values and show the data types 
training_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83624 entries, 0 to 83623
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Request_Date             83624 non-null  datetime64[ns]
 1   Flight_Date              83624 non-null  datetime64[ns]
 2   Duration_Time            83624 non-null  float64       
 3   Departure_hour           83624 non-null  object        
 4   flight_unique_id         83624 non-null  object        
 5   route_abb                83624 non-null  object        
 6   flight_number            83624 non-null  object        
 7   Price                    83624 non-null  float64       
 8   Currency                 83624 non-null  object        
 9   min_future_price         83624 non-null  float64       
 10  price_in_eur             83624 non-null  float64       
 11  min_future_price_in_eur  83624 non-null  float64       
 12  Request_Weekday          83624 n

In [5]:
# import numpy for normalization
import numpy as np

#training_features_log = [['min_future_price_in_eur', 'Duration_Time', 'price_in_eur', 'flight_distance_km']]
#for feature in training_features_log:
#    training_features[feature] = np.log1p(training_features[feature])
    

#testing_features_log = [['Duration_Time', 'price_in_eur', 'flight_distance_km']]
#for feature in testing_features_log:
#    test[feature] = np.log1p(test[feature])

training_features

,Request_Date,Flight_Date,Duration_Time,Departure_hour,flight_unique_id,route_abb,flight_number,Price,Currency,min_future_price,...,min_future_price_in_eur,Request_Weekday,Request_timeofday,Departure_timeofday,flight_distance_km,Request_hour,Request_Date_Month,Flight_Date_Month,route_abb_from,route_abb_to
0,2019-06-03 11:00:00,2019-06-05 19:00:00,56.0,19:00,2019-06-05 FR 146,SXF-STN,FR 146,208.07,€,259.07,...,259.070000,Monday,Morning,Evening,907.2,11:00,6,6,SXF,STN
1,2019-06-03 23:00:00,2019-06-05 19:00:00,44.0,19:00,2019-06-05 FR 146,SXF-STN,FR 146,259.07,€,259.07,...,259.070000,Monday,Night,Evening,907.2,23:00,6,6,SXF,STN
2,2019-06-04 11:00:00,2019-06-05 19:00:00,32.0,19:00,2019-06-05 FR 146,SXF-STN,FR 146,259.07,€,259.07,...,259.070000,Tuesday,Morning,Evening,907.2,11:00,6,6,SXF,STN
3,2019-06-04 23:00:00,2019-06-05 19:00:00,20.0,19:00,2019-06-05 FR 146,SXF-STN,FR 146,259.07,€,259.07,...,259.070000,Tuesday,Night,Evening,907.2,23:00,6,6,SXF,STN
4,2019-06-03 11:00:00,2019-06-05 21:00:00,58.0,21:00,2019-06-05 FR 147,STN-SXF,FR 147,127.49,£,223.37,...,252.059401,Monday,Morning,Night,907.2,11:00,6,6,STN,SXF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83619,2019-08-01 11:00:00,2019-09-10 10:00:00,959.0,10:00,2019-09-10 FR 8543,SXF-STN,FR 8543,35.69,€,39.69,...,39.690000,Thursday,Morning,Morning,907.2,11:00,8,9,SXF,STN
83620,2019-08-01 23:00:00,2019-09-10 10:00:00,947.0,10:00,2019-09-10 FR 8543,SXF-STN,FR 8543,46.83,€,39.69,...,39.690000,Thursday,Night,Morning,907.2,23:00,8,9,SXF,STN
83621,2019-08-02 11:00:00,2019-09-10 10:00:00,935.0,10:00,2019-09-10 FR 8543,SXF-STN,FR 8543,46.83,€,39.69,...,39.690000,Friday,Morning,Morning,907.2,11:00,8,9,SXF,STN
83622,2019-08-02 23:00:00,2019-09-10 10:00:00,923.0,10:00,2019-09-10 FR 8543,SXF-STN,FR 8543,39.69,€,39.69,...,39.690000,Friday,Night,Morning,907.2,23:00,8,9,SXF,STN


# Data preparation
We will now do some minor modifications to put our data into machine-understandable terms. We will use the Python library Pandas for the data manipulation relying on the structure known as dataframe.

The first step is the one-hot-encoding of the data. This process takes categorical variables, such as days of the week and converts it to a numerical representation without an arbitrary ordering.

In [6]:
#from sklearn import preprocessing

#encoder = preprocessing.LabelEncoder()
#train_features['min_future_price_in_eur'] = encoder.fit_transform(train_features['min_future_price_in_eur'])
#train_features['Duration_Time'] = encoder.fit_transform(train_features['Duration_Time'])
#train_features['price_in_eur'] = encoder.fit_transform(train_features['price_in_eur'])
#train_features['flight_distance_km'] = encoder.fit_transform(train_features['flight_distance_km'])

# One-hot encode the data using pandas get_dummies
#train_features = pd.get_dummies(train_features)
#train_features

In [7]:
# One Hot encoding for categorical values
training_features = pd.get_dummies(training_features, columns=['Request_Weekday', 'Request_timeofday', 'Departure_timeofday', 'route_abb_from', 'route_abb_to', 'Departure_hour'])
test = pd.get_dummies(test, columns=['Request_Weekday', 'Request_timeofday', 'Departure_timeofday', 'route_abb_from', 'route_abb_to', 'Departure_hour'])

In [8]:
print('The shape of our features is:', training_features.shape)

The shape of our features is: (83624, 42)


In [9]:
# Descriptive statistics for each column 
training_features.describe()

,Duration_Time,Price,min_future_price,price_in_eur,min_future_price_in_eur,flight_distance_km,Request_Date_Month,Flight_Date_Month,Request_Weekday_Friday,Request_Weekday_Monday,...,route_abb_to_SXF,Departure_hour_10:00,Departure_hour_18:00,Departure_hour_19:00,Departure_hour_21:00,Departure_hour_22:00,Departure_hour_6:00,Departure_hour_7:00,Departure_hour_8:00,Departure_hour_9:00
count,83624.000000,83624.000000,83624.000000,83624.000000,83624.000000,83624.000000,83624.000000,83624.000000,83624.000000,83624.000000,...,83624.000000,83624.000000,83624.000000,83624.000000,83624.000000,83624.000000,83624.000000,83624.000000,83624.000000,83624.000000
mean,929.921147,68.161822,53.703764,71.954198,56.698702,805.252909,6.456125,7.706185,0.142734,0.152468,...,0.316763,0.078375,0.154872,0.240039,0.011026,0.077191,0.212989,0.175117,0.049304,0.001088
std,548.569008,51.927907,43.785672,54.780372,46.164459,137.934369,0.545032,0.826904,0.349804,0.359476,...,0.465217,0.268762,0.361785,0.427109,0.104423,0.266895,0.409422,0.380070,0.216503,0.032970
min,19.000000,8.990000,8.990000,8.990000,8.990000,618.630000,6.000000,6.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,475.000000,30.330000,19.990000,32.760371,22.368936,618.630000,6.000000,7.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,883.000000,52.010000,39.460000,55.070000,42.056918,907.200000,6.000000,8.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1328.000000,88.730000,73.430000,91.790000,73.430000,907.200000,7.000000,8.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2375.000000,334.870000,311.090000,372.956297,311.090000,907.200000,8.000000,9.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


If we look at the dimensions of the data, we notice 83.624 rows which represents the flight requests. We couldn't notice missing data and the data quality seems to be good. There are 42 columns which represent 42 features and the one target ('min_future_price_in_eur').
The summary statistics shows that there are not any data points that immediately appear as anomalous and there are no zeros in any of the measurement columns. By examining the plots and graphs in the file descriptive_analysis.ipynb we can feel confident in the high quality of the data.There are no clear outliers.

In [10]:
# Add train column
test['train'] = 0
training_features['train'] = 1

# Equal columns across both tables
combined = pd.concat([training_features,test], sort=False)
training_features = combined[combined['train'] == 1]
test = combined[combined['train'] == 0]

# Fill NaN with 0
training_features = training_features.fillna(0)
test = test.fillna(0)

# Drop train column
test.drop(['train'], axis=1, inplace=True)
training_features.drop(['train'], axis=1, inplace=True)
test

,Request_Date,Flight_Date,Duration_Time,flight_unique_id,route_abb,flight_number,Price,Currency,min_future_price,price_in_eur,...,route_abb_to_SXF,Departure_hour_10:00,Departure_hour_18:00,Departure_hour_19:00,Departure_hour_21:00,Departure_hour_22:00,Departure_hour_6:00,Departure_hour_7:00,Departure_hour_8:00,Departure_hour_9:00
0,2019-06-03 11:00:00,2019-07-02 07:00:00,692.0,2019-07-02 FR 145,STN-SXF,FR 145,26.87,£,0.0,30.321154,...,1,0,0,0,0,0,0,1,0,0.0
1,2019-06-03 23:00:00,2019-07-02 07:00:00,680.0,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,0.0,44.528200,...,1,0,0,0,0,0,0,1,0,0.0
2,2019-06-04 11:00:00,2019-07-02 07:00:00,668.0,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,0.0,44.467984,...,1,0,0,0,0,0,0,1,0,0.0
3,2019-06-04 23:00:00,2019-07-02 07:00:00,656.0,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,0.0,44.467984,...,1,0,0,0,0,0,0,1,0,0.0
4,2019-06-05 11:00:00,2019-07-02 07:00:00,644.0,2019-07-02 FR 145,STN-SXF,FR 145,39.46,£,0.0,44.521668,...,1,0,0,0,0,0,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5578,2019-07-30 23:00:00,2019-08-02 22:00:00,71.0,2019-08-02 FR 8545,SXF-STN,FR 8545,42.83,€,0.0,42.830000,...,0,0,0,0,0,1,0,0,0,0.0
5579,2019-07-31 11:00:00,2019-08-02 22:00:00,59.0,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,0.0,50.990000,...,0,0,0,0,0,1,0,0,0,0.0
5580,2019-07-31 23:00:00,2019-08-02 22:00:00,47.0,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,0.0,50.990000,...,0,0,0,0,0,1,0,0,0,0.0
5581,2019-08-01 11:00:00,2019-08-02 22:00:00,35.0,2019-08-02 FR 8545,SXF-STN,FR 8545,50.99,€,0.0,50.990000,...,0,0,0,0,0,1,0,0,0,0.0


In the next step we separate the data into the features and targets. The target, also known as the label, is the value we want to predict, in this case the min_future_price and the features are all the columns the model uses to make a prediction. We will also convert the Pandas dataframes to Numpy arrays because that is the way the algorithm works. (I save the column headers, which are the names of the features, to a list to use for later visualization).

In [11]:
# Drop columns we do not need from train
training_features = training_features.drop(columns=['route_abb', 'min_future_price', 'Currency', 'flight_unique_id', 'Price', 'Request_Date', 'Flight_Date', 'Request_Date_Month', 'Flight_Date_Month', 'Request_hour', 'flight_number'])

# Drop columns we do not need from test
test = test.drop(columns=['route_abb', 'min_future_price', 'Currency', 'flight_unique_id', 'Price', 'min_future_price', 'min_future_price_in_eur', 'Request_Date', 'Flight_Date', 'Request_Date_Month', 'Flight_Date_Month', 'Request_hour', 'flight_number'])

In [12]:
# Use numpy to convert to arrays
# Labels are the values we want to predict
labels = np.array(training_features['min_future_price_in_eur'])

# Remove the labels from the features
# axis 1 refers to the columns
training_features = training_features.drop('min_future_price_in_eur', axis = 1)

# Saving feature names for later use
feature_list = list(training_features.columns)

# Convert to numpy array
training_features = np.array(training_features)

# Splitting data into training and testing sets
During training, we let the model ‘see’ the answers, in this case the min_future_price_inr_eur, so it can learn how to predict the price from the features. We expect there to be some relationship between all the features and the target value, and the model’s job is to learn this relationship during training. 
I am setting the random state to 42 which means the results will be the same each time I run the split for reproducible results.

The following code splits the data sets with another single line:

In [13]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(training_features, labels, test_size = 0.25, random_state = 42)

We look at the shape of all the data to make sure we did everything correctly. The training features number of columns match the testing feature number of columns and the number of rows match for the respective training and testing features and the labels:

In [15]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (62718, 30)
Training Labels Shape: (62718,)
Testing Features Shape: (20906, 30)
Testing Labels Shape: (20906,)


Before we can make and evaluate predictions, we need to establish a baseline. If our model cannot improve upon the baseline, then it will be a failure and we should try a different model. The baseline prediction for our case can be the historical min future price averages. In other words, our baseline is the error we would get if we simply predicted the average min future price for all flight requests.

In [16]:
# The baseline predictions are the historical averages
#baseline_preds = test_features[:, feature_list.index(average_min_future_price)]

# Baseline errors, and display average baseline error
#baseline_errors = abs(baseline_preds - test_labels)

#print('Average baseline error: ', round(np.mean(baseline_errors), 2))

# Train the model

In [18]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 50 decision trees
rf = RandomForestRegressor(n_estimators = 40, random_state = 42)

# Train the model on training data
rf.fit(train_features, train_labels)
rf.score(test_features, test_labels)

0.8730030186739571

In [19]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [20]:
def get_score(model, train_features, test_features, train_labels, test_labels):
    model.fit(train_features, train_labels)
    return model.score(test_features, test_labels)

In [21]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=3)

In [22]:
scores_rf = []

for train_index, test_index in kf.split(training_features):
    train_features, test_features, train_labels, test_labels = training_features[train_index], training_features[test_index], labels[train_index], labels[test_index]
    scores_rf.append(get_score(RandomForestRegressor(n_estimators=40), train_features, test_features, train_labels, test_labels))

In [23]:
scores_rf

[0.7410091988800555, 0.7614847331621946, 0.7141632249625798]

In [24]:
from sklearn.model_selection import cross_val_score

# using cross validation score
cross_val_score(RandomForestRegressor(n_estimators=40), training_features, labels)

array([0.68971711, 0.76205439, 0.74206152, 0.72549359, 0.77653767])

# Make Predictions on the Test Set
Our model has now been trained to learn the relationships between the features and the targets. The next step is figuring out how good the model is! To do this we make predictions on the test features. We then compare the predictions to the known answers. When performing regression, we need to make sure to use the absolute error because we expect some of our answers to be low and some to be high. We are interested in how far away our average prediction is from the actual value so we take the absolute value (as we also did when establishing the baseline).

In [25]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 2.93 degrees.


# Determine Performance Metrics
To put our predictions in perspective, we can calculate an accuracy using the mean average percentage error subtracted from 100 %.

In [26]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 92.45 %.


Our model has learned how to predict the min_future_price for the requested flights with 92.45% accuracy. 

# Interpret model 
The following code takes one tree from the forest and saves it as an image.

In [27]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz 
import pydot

# Pull out one tree from the forest
#tree = rf.estimators_[5]

# Export the image to a dot file
#export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# Use dot file to create a graph
#(graph, ) = pydot.graph_from_dot_file('tree.dot')

# Write graph to a png file
#graph.write_png('tree.png')

In [28]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)

# Extract the small tree
tree_small = rf_small.estimators_[5]

# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');

# Variable importance
In order to quantify the usefulness of all the variables in the entire random forest, we can look at the relative importances of the variables. The importances returned in Skicit-learn represent how much including a particular variable improves the prediction. 

In [ ]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

At the top of the list is price_in_eur, the price of the flight on the requested date. This tells us the best predictor of the min_future_price is the requested flight price. 

The second most important factor is the duration time between request and flight date.

In future implementations of the model, we can remove those variables that have no importance and the performance will not suffer. Let’s make a random forest with only the two most important variables, the price_in_eur and the duration_time and see how the performance compares.

In [ ]:
# New random forest with only the two most important variables
rf_most_important = RandomForestRegressor(n_estimators= 50, random_state=42)

# Extract the two most important features
important_indices = [feature_list.index('price_in_eur'), feature_list.index('Duration_Time')]
train_important = train_features[:, important_indices]
test_important = test_features[:, important_indices]

# Train the random forest
rf_most_important.fit(train_important, train_labels)

# Make predictions and determine the error
predictions = rf_most_important.predict(test_important)
errors = abs(predictions - test_labels)

# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')

# Visualizations
A simple bar plot of the feature importances to illustrate the disparities in the relative significance of the variables.
Next, we plot the entire dataset with predictions highlighted. We can use this plot to determine if there are any outliers in either the data or our predictions.

In [ ]:
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt
%matplotlib inline

# Set the style
plt.style.use('fivethirtyeight')

# list of x locations for plotting
x_values = list(range(len(importances)))

# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')

# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')

# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');